In [2]:
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path

In [36]:
import xarray as xr

In [27]:
from typing import Union, Dict, Tuple, List


def xml_parameters(xml: Union[str, Path], pars: Union[str, List[str]] = None) -> Dict:
    """It extracts the temporal information from the settings XML file.
    
    Input:
    ------
    xml:         Union[str, Path] 
        A XML settings file (path, filename and extension)
    pars:        Union[str, List[str]]
        Name(s) of the parameters to be extracted
        
    Output:
    -------
    parameters:  Dict
        Keys are parameter names and values the calibrated parameter value
    """
    
    # extract temporal info from the XML
    tree = ET.parse(xml)
    root = tree.getroot()
    
    if pars is None:
        pars = ['b_Xinanjiang', 'UpperZoneTimeConstant', 'LowerZoneTimeConstant', 'LZThreshold',
                'GwPercValue', 'GwLoss', 'PowerPrefFlow', 'SnowMeltCoef',
                'AvWaterRateThreshold' , 'LakeMultiplier', 'adjust_Normal_Flood', 'ReservoirRnormqMult', 
                'QSplitMult', 'CalChanMan', 'CalChanMan2', 'ChanBottomWMult', 'ChanDepthTMult', 'ChanSMult']
    
    parameters = {par: float(root.find(f'.//textvar[@name="{par}"]').attrib['value']) for par in pars}
        
    return parameters

## Configuration

In [3]:
path_glofas = Path('Z:/nahaUsers/casadje/GloFASv4/')
settings_file = 'settings_lisflood_wusewregion_FULLinit_GloFASnext-PreRunX.xml'

## Data
### Calibrated parameters

In [5]:
catchments = [x for x in os.listdir(path_glofas / 'catchments') if os.path.isdir(os.path.join(path_glofas / f'catchments/{x}'))]

len(catchments)

33

In [33]:
# read calibrated parameters
parameters = {}
for catchment in catchments:
    xml = path_glofas / 'catchments' / catchment / 'settings' / settings_file
    parameters[catchment] = xml_parameters(xml, pars=['adjust_Normal_Flood', 'ReservoirRnormqMult'])
parameters = pd.DataFrame(parameters)

parameters

,453,476,477,527,555,556,635,642,656,662,...,825,829,830,831,835,836,854,856,858,865
adjust_Normal_Flood,0.139326,0.522563,0.638480,0.129179,0.501281,0.894556,0.146777,0.01000,0.026621,0.480791,...,0.325858,0.609061,0.958427,0.223297,0.544355,0.617579,0.387440,0.185101,0.982214,0.474467
ReservoirRnormqMult,0.250000,1.999610,0.828612,1.998290,0.946985,1.714180,1.999660,1.59126,1.999900,1.256110,...,1.843080,0.385225,0.453351,0.987671,0.372642,1.205480,0.705504,2.000000,0.451097,0.954613


In [51]:
# identify catchments in the catchment
reservoirs = {}
for catchment in catchments:
    try:
        # read NetCDF file
        nc = path_glofas / 'catchments' / catchment / 'maps' / '20210920_res_Global_03min.nc'
        res = xr.open_dataset(nc)['res'].data
        # extract unique ResID values
        reservoirs[catchment] = res[~np.isnan(res)].astype(int).tolist()
    except:
        print(catchment)

741


In [52]:
reservoirs

{'453': [31, 169],
 '476': [609],
 '477': [169, 364],
 '527': [617],
 '555': [48],
 '556': [227, 14],
 '635': [],
 '642': [146],
 '656': [621, 488, 54],
 '662': [193, 195],
 '695': [228],
 '748': [7],
 '750': [92],
 '755': [50],
 '772': [362, 149],
 '788': [76, 56],
 '789': [386],
 '800': [446, 334],
 '807': [392, 140, 278],
 '809': [308, 334],
 '811': [56, 313],
 '820': [143, 17],
 '825': [446, 462, 258],
 '829': [313, 406],
 '830': [491],
 '831': [392, 140, 16, 174],
 '835': [362, 149, 446, 308, 334, 462, 258, 313, 359, 406, 449],
 '836': [359],
 '854': [442],
 '856': [315],
 '858': [393, 64],
 '865': [156]}

In [59]:
for key, ls in reservoirs.items():
    if 12 in ls:
        print(f'Reservoir 12 is in catchment {key}')
    elif 621 in ls:
        print(f'Reservoir 621 is in catchment {key}')

Reservoir 621 is in catchment 656


In [57]:
ids = np.unique([id for key, ls in reservoirs.items() for id in ls])
print(f'no. reservoirs: {len(ids)}')

no. reservoirs: 45
